# Base
> baseclass for metric

In [10]:
#| default_exp metric.base

In [1]:
#| hide
from dotenv import load_dotenv
load_dotenv('/Users/shahules/Myprojects/ragas_annotator/.envrc')

True

In [ ]:
#| export
import typing as t
from typing import Any, Callable, Dict, List, Optional, Union
from abc import ABC, abstractmethod
import asyncio
from dataclasses import dataclass
from pydantic import BaseModel
import openai
import instructor
from dataclasses import dataclass, field
from pydantic import BaseModel, create_model
import typing as t
import inspect

### MetricResult
> Class to hold the result metric

In [1]:
import typing as t
from typing import Any, Callable, Dict, List, Optional, Union

class MetricResult:
    """Class to hold the result of a metric evaluation.
    
    This class behaves like its underlying result value but still provides access
    to additional metadata like reasoning.
    
    Works with:
    - DiscreteMetrics (string results)
    - NumericMetrics (float/int results)
    - RankingMetrics (list results)
    """
    
    def __init__(self, result: Any, reason: t.Optional[str] = None):
        self._result = result
        self.reason = reason
    
    def __repr__(self):
        return repr(self._result)
    
    # Access to underlying result
    @property
    def result(self):
        """Get the raw result value."""
        return self._result
    
    
    # String conversion - works for all types
    def __str__(self):
        return str(self._result)
    
    # Container-like behaviors for list results (RankingMetric)
    def __getitem__(self, key):
        if not hasattr(self._result, "__getitem__"):
            raise TypeError(f"{type(self._result).__name__} object is not subscriptable")
        return self._result[key]
    
    def __iter__(self):
        if not hasattr(self._result, "__iter__"):
            raise TypeError(f"{type(self._result).__name__} object is not iterable")
        return iter(self._result)
    
    def __len__(self):
        if not hasattr(self._result, "__len__"):
            raise TypeError(f"{type(self._result).__name__} has no len()")
        return len(self._result)
    
    # Numeric operations for numeric results (NumericMetric)
    def __float__(self):
        if isinstance(self._result, (int, float)):
            return float(self._result)
        raise TypeError(f"Cannot convert {type(self._result).__name__} to float")
    
    def __int__(self):
        if isinstance(self._result, (int, float)):
            return int(self._result)
        raise TypeError(f"Cannot convert {type(self._result).__name__} to int")
    
    def __add__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot add {type(self._result).__name__} objects")
        if isinstance(other, MetricResult):
            return self._result + other._result
        return self._result + other
    
    def __radd__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot add {type(self._result).__name__} objects")
        return other + self._result
    
    def __sub__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot subtract {type(self._result).__name__} objects")
        if isinstance(other, MetricResult):
            return self._result - other._result
        return self._result - other
    
    def __rsub__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot subtract {type(self._result).__name__} objects")
        return other - self._result
    
    def __mul__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot multiply {type(self._result).__name__} objects")
        if isinstance(other, MetricResult):
            return self._result * other._result
        return self._result * other
    
    def __rmul__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot multiply {type(self._result).__name__} objects")
        return other * self._result
    
    def __truediv__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot divide {type(self._result).__name__} objects")
        if isinstance(other, MetricResult):
            return self._result / other._result
        return self._result / other
    
    def __rtruediv__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot divide {type(self._result).__name__} objects")
        return other / self._result
    
    # Comparison operations - work for all types with same-type comparisons
    def __eq__(self, other):
        if isinstance(other, MetricResult):
            return self._result == other._result
        return self._result == other
    
    def __lt__(self, other):
        if isinstance(other, MetricResult):
            return self._result < other._result
        return self._result < other
    
    def __le__(self, other):
        if isinstance(other, MetricResult):
            return self._result <= other._result
        return self._result <= other
    
    def __gt__(self, other):
        if isinstance(other, MetricResult):
            return self._result > other._result
        return self._result > other
    
    def __ge__(self, other):
        if isinstance(other, MetricResult):
            return self._result >= other._result
        return self._result >= other
    
    # Method forwarding for type-specific behaviors
    def __getattr__(self, name):
        """Forward attribute access to the result object if it has that attribute.
        
        This allows calling string methods on discrete results, 
        numeric methods on numeric results, and list methods on ranking results.
        """
        if hasattr(self._result, name):
            attr = getattr(self._result, name)
            if callable(attr):
                # If it's a method, wrap it to return MetricResult when appropriate
                def wrapper(*args, **kwargs):
                    result = attr(*args, **kwargs)
                    # If the result is of the same type as self._result, wrap it
                    if isinstance(result, type(self._result)):
                        return MetricResult(result=result, reason=self.reason)
                    return result
                return wrapper
            return attr
        raise AttributeError(f"{type(self).__name__} has no attribute '{name}'")
    
    # JSON/dict serialization
    def to_dict(self):
        """Convert the result to a dictionary."""
        return {
            "result": self._result,
            "reason": self.reason
        }

In [9]:
result = MetricResult(result='low',reason="my reason")
result


'low'

In [11]:
result.reason

'my reason'

In [ ]:
@dataclass
class LLM:

    def __post_init__(self):
        self.aclient = instructor.from_openai(openai.AsyncOpenAI())
        self.client = instructor.from_openai(openai.OpenAI())

    
    def generate(self,prompt,response_model):
        return self.client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt},
        ],
        response_model=response_model,
    )

    async def agenerate(self,prompt,response_model):
        return await self.aclient.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt},
        ],
        response_model=response_model,
    )

### Metric

In [12]:
#| export

@dataclass
class Metric(ABC):
    """Base class for all metrics in the LLM evaluation library."""
    name:str
    prompt:str
    llm:LLM
    _response_models: Dict[bool, Type[BaseModel]] = field(
        default_factory=dict, init=False, repr=False
    )
    
    @abstractmethod
    def _get_response_model(self, with_reasoning: bool) -> Type[BaseModel]:
        """Get the appropriate response model."""
        pass
    
    def score(self, reasoning=True, n=1, **kwargs) -> Any:
        
        prompt_input = self.prompt.format(**kwargs)
        response = self.llm.generate(prompt_input, response_model = self._get_response_model(reasoning))
        return MetricResult(**response.model_dump())

    async def ascore(self, reasoning=True, n=1, **kwargs):
        prompt_input = self.prompt.format(**kwargs)
        response = await self.llm.agenerate(prompt_input, response_model = self._get_response_model(reasoning))
        return MetricResult(**response.model_dump())
        
    def batch_score(self, inputs: List[Dict[str, Any]], reasoning:bool=True, n:int=1) -> List[Any]:
       
        return [self.score(**input_dict) for input_dict in inputs]
    
    async def abatch_score(self, inputs: List[Dict[str, Any]], reasoning: bool = True, n: int = 1) -> List[MetricResult]:
        
        async_tasks = []
        for input_dict in inputs:
            # Add reasoning and n to the input parameters
            async_tasks.append(self.ascore(reasoning=reasoning, n=n, **input_dict))
            
        # Run all tasks concurrently and return results
        return await asyncio.gather(*async_tasks)





NameError: name 'dataclass' is not defined

### DiscreteMetric 


In [9]:
#| export

@dataclass
class DiscreteMetrics(Metric):
    values: t.List[str] = field(default_factory=lambda: ["pass", "fail"])
    _response_models: t.Dict[bool, t.Type[BaseModel]] = field(default_factory=dict, init=False, repr=False)
    
    def _get_response_model(self, with_reasoning: bool) -> t.Type[BaseModel]:
        """Get or create a response model based on reasoning parameter."""
        
        if with_reasoning in self._response_models:
            return self._response_models[with_reasoning]
        
        model_name = 'response_model'
        fields = {"score": (t.Literal[tuple(self.values)], ...)}
        
        if with_reasoning:
            fields["reason"] = (str, ...)
        
        model = create_model(model_name, **fields)
        self._response_models[with_reasoning] = model
        return model 


    

### decorator factory for discrete_metric

In [15]:
#| export


def discrete_metric(llm, prompt, values:t.List[str],name:t.Optional[str]=None):

    def decorator(metric):
        metric_name = name or metric.__name__ 
        is_async = inspect.iscoroutinefunction(metric)

        @dataclass
        class CustomDiscreteMetric(DiscreteMetrics):

            def score(self,reasoning:bool=True, n:int=1,**kwargs):

                if is_async:
                    # For async functions, we need to run them in an event loop
                    import asyncio
                    
                    # Get or create an event loop
                    try:
                        loop = asyncio.get_event_loop()
                    except RuntimeError:
                        loop = asyncio.new_event_loop()
                        asyncio.set_event_loop(loop)
                    
                    # Run the async function and get the result
                    result = loop.run_until_complete(metric(self.llm,self.prompt,**kwargs))
                else:
                    # For sync functions, just call directly
                    result = metric(self.llm,self.prompt,**kwargs)
                    
                if isinstance(result, tuple) and len(result) == 2:
                    score, reason = result
                else:
                    score = result
                    reason = None
                    
                return MetricResult(score=score, reason=reason if reasoning else None)
            
            async def ascore(self, reasoning=True, n=1, **kwargs):
                if is_async:
                    # For async functions, await them directly
                    result = await metric(self.llm,self.prompt,**kwargs)
                else:
                    # For sync functions, run them normally
                    result = metric(self.llm,self.prompt,**kwargs)

                if isinstance(result, tuple) and len(result) == 2:
                    score, reason = result
                else:
                    score = result
                    reason = None
                    
        metric_instance = CustomDiscreteMetric(
            name=metric_name,
            prompt=prompt,
            llm=llm,
            values=values,
            
        )
        metric_instance.__name__ = name
        metric_instance.__doc__ = metric.__doc__
        
        return metric_instance
    
    return decorator
            
    

            

### Usage pattern

In [40]:
my_metric = DiscreteMetrics(
    name='helpfulness',
    llm=LLM(),
    prompt="Evaluate if given answer is helpful\n\n{response}",
    values=["low","med","high"],
)

result = my_metric.score(response="this is my response")
result #gives "low"
result.reason #gives reasoning from llm



In [18]:
## score without reasoning to save reasoning tokens cost
result = my_metric.score(response="this is my response",reasoning=False)
result

MetricResult(score=low, reason=None)

In [24]:
@discrete_metric(llm=LLM(),
    prompt="Evaluate if given answer is helpful\n\n{response}",
    name='new_metric',values=["low","med","high"])
def my_metric(llm,prompt,**kwargs):

        class response_model(BaseModel):
             output: t.List[bool]
             reason: str
        
        response = llm.generate(prompt.format(**kwargs),response_model=response_model)
        total = sum(response.output)
        if total < 1:
            score = 'low'
        else:
            score = 'high'
        return score,"reason"

result = my_metric.score(response='my response') # result
result.score
result.reason

'reason'

In [25]:
result

MetricResult(score=high, reason=reason)

## Numeric Metric

In [29]:
#| export

@dataclass
class NumericMetrics(Metric):
    range: t.Tuple[float,float]
    _response_models: t.Dict[bool, t.Type[BaseModel]] = field(default_factory=dict, init=False, repr=False)
    
    def _get_response_model(self, with_reasoning: bool) -> t.Type[BaseModel]:
        """Get or create a response model based on reasoning parameter."""
        
        if with_reasoning in self._response_models:
            return self._response_models[with_reasoning]
        
        model_name = 'response_model'
        fields = {"score": (float,...)}
        
        if with_reasoning:
            fields["reason"] = (str, ...)
        
        model = create_model(model_name, **fields)
        self._response_models[with_reasoning] = model
        return model 
    
    def score(self, reasoning=True, n=1, **kwargs) -> Any:
        
        prompt_input = self.prompt.format(**kwargs)
        response = self.llm.generate(prompt_input, response_model = self._get_response_model(reasoning))
        return MetricResult(**response.model_dump())

    async def ascore(self, reasoning=True, n=1, **kwargs):
        prompt_input = self.prompt.format(**kwargs)
        response = await self.llm.agenerate(prompt_input, response_model = self._get_response_model(reasoning))
        return MetricResult(**response.model_dump())



In [30]:
my_metric = NumericMetrics(
    name='helpfulness',
    llm=LLM(),
    prompt="Evaluate if given answer is helpful\n\n{response}",
    range=(0,10),
)

result = my_metric.score(response="this is my response")
result #gives "low"
result.reason #gives reasoning from llm

result = my_metric.batch_score(inputs=[{"response":"this is my response"}])


### decorator factory

In [37]:
#| export


def numeric_metric(llm, prompt, range: t.Tuple[float,float],name:t.Optional[str]=None):

    def decorator(metric):
        metric_name = name or metric.__name__ 
        is_async = inspect.iscoroutinefunction(metric)

        @dataclass
        class CustomNumericMetric(NumericMetrics):

            def score(self,reasoning:bool=True, n:int=1,**kwargs):

                if is_async:
                    # For async functions, we need to run them in an event loop
                    import asyncio
                    
                    # Get or create an event loop
                    try:
                        loop = asyncio.get_event_loop()
                    except RuntimeError:
                        loop = asyncio.new_event_loop()
                        asyncio.set_event_loop(loop)
                    
                    # Run the async function and get the result
                    result = loop.run_until_complete(metric(self.llm,self.prompt,**kwargs))
                else:
                    # For sync functions, just call directly
                    result = metric(self.llm,self.prompt,**kwargs)
                    
                if isinstance(result, tuple) and len(result) == 2:
                    score, reason = result
                else:
                    score = result
                    reason = None
                    
                return MetricResult(score=score, reason=reason if reasoning else None)
            
            async def ascore(self, reasoning=True, n=1, **kwargs):
                if is_async:
                    # For async functions, await them directly
                    result = await metric(self.llm,self.prompt,**kwargs)
                else:
                    # For sync functions, run them normally
                    result = metric(self.llm,self.prompt,**kwargs)

                if isinstance(result, tuple) and len(result) == 2:
                    score, reason = result
                else:
                    score = result
                    reason = None
                    
        metric_instance = CustomNumericMetric(
            name=metric_name,
            prompt=prompt,
            llm=llm,
            range=range,
            
        )
        metric_instance.__name__ = name
        metric_instance.__doc__ = metric.__doc__
        
        return metric_instance
    
    return decorator
            
    

            

In [38]:
@numeric_metric(llm=LLM(),
    prompt="Evaluate if given answer is helpful\n\n{response}",
    name='new_metric',range=(0,10))
def my_metric(llm,prompt,**kwargs):

        class response_model(BaseModel):
             output: int
             reason: str
        
        response = llm.generate(prompt.format(**kwargs),response_model=response_model)
        total = response.output
        if total < 1:
            score = 0
        else:
            score = 10
        return score,"reason"

result = my_metric.score(response='my response') # result
result # 10
result.reason # the reason for the answer

result1 = my_metric.score(response='my response 1') # result
result2 = my_metric.score(response='my response 2') # result

result1 + result2 # should be addable and behave like a float


'reason'

In [39]:
result.score


0

### Ranking metric

In [49]:
#| export

@dataclass
class RankingMetrics(Metric):
    num_ranks: int
    _response_models: t.Dict[bool, t.Type[BaseModel]] = field(default_factory=dict, init=False, repr=False)
    
    def _get_response_model(self, with_reasoning: bool) -> t.Type[BaseModel]:
        """Get or create a response model based on reasoning parameter."""
        
        if with_reasoning in self._response_models:
            return self._response_models[with_reasoning]
        
        model_name = 'response_model'
        fields = {"score": (t.List[int],...)}
        
        if with_reasoning:
            fields["reason"] = (str, ...)
        
        model = create_model(model_name, **fields)
        self._response_models[with_reasoning] = model
        return model 
    
    def score(self, reasoning=True, n=1, **kwargs) -> Any:
        
        prompt_input = self.prompt.format(**kwargs)
        response = self.llm.generate(prompt_input, response_model = self._get_response_model(reasoning))
        return MetricResult(**response.model_dump())

    async def ascore(self, reasoning=True, n=1, **kwargs):
        prompt_input = self.prompt.format(**kwargs)
        response = await self.llm.agenerate(prompt_input, response_model = self._get_response_model(reasoning))
        return MetricResult(**response.model_dump())



In [50]:
# User instantiates a ranking metric by providing a name, an LLM, a prompt template, and the number of rankings desired.
my_ranking_metric = RankingMetrics(
    name='response_ranking',
    llm=LLM(),  # Your language model instance
    prompt="Rank the following responses:\n{candidates}",
    num_ranks=3
)

# To score a single input (ranking candidate responses)
result = my_ranking_metric.score(candidates=[
    "short answer.",
    "a bit more detailed.",
    "the longest and most detailed answer."
])
print(result.score)   # Might output something like: [1, 0, 2]
print(result.reason)  # Provides the reasoning behind the ranking


[1, 2, 3]
The responses are ranked based on their length and detail, with the longest and most detailed answer receiving the highest rank.


### decorator factory for ranking metric

In [51]:
#| export

def ranking_metric(llm, prompt, num_ranks: int, name: t.Optional[str] = None):
    def decorator(metric):
        metric_name = name or metric.__name__
        is_async = inspect.iscoroutinefunction(metric)

        @dataclass
        class CustomRankingMetric(RankingMetrics):
            # Inherits: name, prompt, llm, num_ranks from RankingMetrics.
            # No extra fields are needed.
            def score(self, reasoning: bool = True, n: int = 1, **kwargs):
                if is_async:
                    # For async functions, run in an event loop.
                    import asyncio
                    try:
                        loop = asyncio.get_event_loop()
                    except RuntimeError:
                        loop = asyncio.new_event_loop()
                        asyncio.set_event_loop(loop)
                    result = loop.run_until_complete(metric(self.llm, self.prompt, **kwargs))
                else:
                    result = metric(self.llm, self.prompt, **kwargs)

                if isinstance(result, tuple) and len(result) == 2:
                    ranking, reason = result
                else:
                    ranking = result
                    reason = None

                return MetricResult(score=ranking, reason=reason if reasoning else None)

            async def ascore(self, reasoning: bool = True, n: int = 1, **kwargs):
                if is_async:
                    result = await metric(self.llm, self.prompt, **kwargs)
                else:
                    result = metric(self.llm, self.prompt, **kwargs)

                if isinstance(result, tuple) and len(result) == 2:
                    ranking, reason = result
                else:
                    ranking = result
                    reason = None

                return MetricResult(score=ranking, reason=reason if reasoning else None)

        metric_instance = CustomRankingMetric(
            name=metric_name,
            prompt=prompt,
            llm=llm,
            num_ranks=num_ranks
        )
        metric_instance.__name__ = metric_name
        metric_instance.__doc__ = metric.__doc__

        return metric_instance
    return decorator

In [52]:
@ranking_metric(
    llm=LLM(),  # Your language model instance
    prompt="Rank the following responses:\n{candidates}",
    name='new_ranking_metric',
    num_ranks=3
)
def my_ranking_metric(llm, prompt, **kwargs):
    # Your custom logic that calls the LLM and returns a tuple of (ranking, reason)
    # For example, process the prompt (formatted with candidates) and produce a ranking.
    ranking = [1, 0, 2]  # Dummy ranking: second candidate is best, then first, then third.
    reason = "Ranked based on response clarity and detail."
    return ranking, reason

# Using the decorator-based ranking metric:
result = my_ranking_metric.score(candidates=[
    "Response A: short answer.",
    "Response B: a bit more detailed.",
    "Response C: the longest and most detailed answer."
])
print(result.score)   # E.g., [1, 0, 2]
print(result.reason)  # E.g., "Ranked based on response clarity and detail."


[1, 0, 2]
Ranked based on response clarity and detail.
